In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import openpyxl
from datetime import datetime

# Função para extrair informações do site e salvar na planilha
def extrair_informacoes(link):
    requisicao = requests.get(link)
    site = BeautifulSoup(requisicao.text, "html.parser")
    
    nome_produto = site.find(class_='product-meta__title heading h3')

    preco_elemento = site.find('span', class_='price--large')
    price_text = preco_elemento.get_text().replace(',', '.').replace('$', '').strip() if preco_elemento else None

    # Verificar se o preço inclui "Sale price" e remover, se presente
    if price_text and price_text.startswith('Sale price'):
        price_text = price_text.replace('Sale price', '').strip()

    # Remover caracteres indesejados e formatar o preço
    if price_text:
        preco_produto = price_text.replace(',', '').replace('.', '').strip()
        # Forçar o tipo de dado para número
        preco_produto = float(preco_produto) if preco_produto else None
    else:
        preco_produto = None
    
    sku_tag = site.find('span', class_='product-meta__sku-number')
    if sku_tag:
        sku_produto = sku_tag.get_text()
    else:
        sku_produto = None

    div_thumbnail = site.find('div', class_='product__thumbnail')
    if div_thumbnail:
        img_tag = div_thumbnail.find('img')
        if img_tag:
            image_link = img_tag['src']
        else:
            image_link = None
    else:
        image_link = None
    
    # Encontrando a próxima linha vazia na planilha
    proxima_linha = sheet.max_row + 1
    
    # Salvando as informações na planilha na próxima linha vazia
    sheet[f'A{proxima_linha}'] = nome_produto.text if nome_produto else ''
    sheet[f'B{proxima_linha}'] = preco_produto if preco_produto is not None else ''
    sheet[f'C{proxima_linha}'] = sku_produto if sku_produto else ''
    sheet[f'D{proxima_linha}'] = image_link
    sheet[f'F{proxima_linha}'] = 'DS18 USA'
    
    # Adicionando a data na próxima linha vazia
    sheet[f'E{proxima_linha}'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Definir o formato da célula como número
    sheet[f'B{proxima_linha}'].number_format = '#,##0.00'

# Lendo os links da planilha Excel
planilha = pd.read_excel('03.Input_links_ds18.xlsx')

# Carregando a planilha de produtos existente
workbook = openpyxl.load_workbook('02.Output_informacoes_produtos_ds18.xlsx')
sheet = workbook.active

# Verificando se as colunas já existem na planilha
if sheet['A1'].value is None or 'Nome do Produto' not in sheet['A1'].value:
    sheet['A1'] = 'Nome do Produto'
if sheet['B1'].value is None or 'Preço do Produto' not in sheet['B1'].value:
    sheet['B1'] = 'Preço do Produto'
if sheet['C1'].value is None or 'SKU do Produto' not in sheet['C1'].value:
    sheet['C1'] = 'SKU do Produto'
if sheet['D1'].value is None or 'Imagem do Produto' not in sheet['D1'].value:
    sheet['D1'] = 'Imagem do Produto'
if sheet['E1'].value is None or 'Data da Extração' not in sheet['E1'].value:
    sheet['E1'] = 'Data da Extração'
if sheet['F1'].value is None or 'Origem' not in sheet['F1'].value:
    sheet['F1'] = 'Origem'

# Iterando sobre cada linha da planilha de links
for idx, row in planilha.iterrows():
    link = row['Link']
    extrair_informacoes(link)  # Removido o parâmetro linha
    
# Salvando a planilha
workbook.save('02.Output_informacoes_produtos_ds18.xlsx')


ChunkedEncodingError: ('Connection broken: IncompleteRead(4101 bytes read, 6139 more expected)', IncompleteRead(4101 bytes read, 6139 more expected))